<a href="https://colab.research.google.com/github/GabrielaCuervoR/AnaliticaMarketing/blob/main/c_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
movies=pd.read_sql("""select * from movies""", conn)
genres=movies['genres'].str.split('|')
te = TransactionEncoder()
genres = te.fit_transform(genres)
genres = pd.DataFrame(genres, columns = te.columns_)

In [ ]:
genres= genres.drop(['(no genres listed)'], axis=1)

In [ ]:
genres4= genres.copy()

In [ ]:
##### Convertir los argumentos true y false en 1 y 0
genres4 = genres4.replace({True: 1, False: 0})

In [ ]:
genres4 = genres4.astype(int)

In [ ]:
genres4.insert(0, 'movieId', movies['movieId'])
genres4.insert(1, 'title', movies['title'])

In [ ]:
# genres_dum2=pd.get_dummies(genres,columns=genres.columns)
# genres_dum2.insert(0, 'movieId', movies['movieId'])
# genres_dum2.insert(1, 'title', movies['title'])

In [ ]:
movies3=genres_dum2
# movies2=genres4

In [ ]:
# movies2.to_sql('movies2',conn, if_exists='replace' )
movies3.to_sql('movies3',conn, if_exists='replace' )

In [ ]:
# ##Convertir columna timestamp a formato fecha
# Date = pd.read_sql("""SELECT
#             strftime('%Y-%m-%d', timestamp, 'unixepoch') AS fecha FROM ratings""", conn)
##Convertir columna timestamp a formato fecha
Date1 = pd.read_sql("""SELECT
            strftime('%Y', timestamp, 'unixepoch') AS fecha FROM ratings""", conn)

In [ ]:
ratings3=ratings.copy()

In [ ]:
ratings3['fecha']= Date1 ##Agregar columna con la fecha a la base ratings

In [ ]:
ratings3=ratings3.drop('timestamp',axis=1)

In [ ]:
ratings3.to_sql('ratings3',conn, if_exists='replace' )

In [ ]:
##Calcular la distribución de las calificaciones
pd.read_sql("""select a.title, count(b.rating) as cnt
            from movies a left join ratings b on a.movieId=b.movieId
            where b.rating group by a.title order by cnt asc """, conn)

In [ ]:
base_nueva1 = pd.read_sql("""select a.*,b.userId,b.rating,b.fecha
            from movies3 a left join ratings3 b on a.movieId=b.movieId
            """, conn)

In [ ]:
base_nueva1.to_sql('base_nueva1',conn, if_exists='replace' )

In [ ]:
##### Descripción base de ratings
###calcular la distribución de calificaciones
cr=pd.read_sql(""" select
                          "ratings" as rating,
                          count(*) as conteo
                          from ratings
                          group by "rating"
                          order by conteo desc""", conn)

In [ ]:
#### filtrar usuarios con más de 50 peliculas calificadas (para tener calificaion confiable) y los que tienen mas de mil porque pueden ser no razonables
pelisrating=pd.read_sql(''' select "userId" as user_id,
                         count(*) as cnt_rat
                         from ratings
                         group by "user_id"
                         having cnt_rat >=30 and cnt_rat <=1000
                         order by cnt_rat asc
                         ''',conn )

In [ ]:
ratingtitle= pd.read_sql("""select b.*,a.title
            from ratings2 b left join movies a on b.movieId=a.movieId
            """, conn)

In [ ]:
ratingtitle =ratingtitle.drop(['index'], axis=1)

In [ ]:
ratingtitle.to_sql('ratingtitle',conn, if_exists='replace' )

In [ ]:
ratingtitle.loc[ratingtitle['title']== 'World Is Not Enough, The (1999)'].value_counts()

In [ ]:
#### filtrar pelicas con más de 30 calificaciones (para tener calificaion confiable) y los que tienen menos de 500 porque pueden ser no razonables
pelrating=pd.read_sql(''' select  "title" as title,
                         count(*) as cnt_val
                         from ratingtitle
                         group by "title"
                         having cnt_val >=30 and cnt_val <=500
                         order by cnt_val asc
                        #  ''',conn )

In [ ]:
######################################################################
################## 1. sistemas basados en popularidad ###############
#####################################################################

In [ ]:
##### recomendaciones basado en popularidad ######

#### mejores peliculas calificadas que tengan una calificación diferente de 0.0
pd.read_sql("""select title,
            avg(rating) as avg_rat,
            count(*) as watch_num
            from ratingtitle
            where rating<>0.0
            group by title
            having watch_num >=15 and watch_num <=100
            order by avg_rat desc
            limit 10
            """, conn)

In [ ]:
#### las peliculas mas vistas con promedio de los que calficaron ###
pd.read_sql("""select title,
            avg(iif(rating = 0, Null, rating)) as avg_cal,
            count(*) as watch_num
            from ratingtitle
            group by title
            order by watch_num desc
            limit 10
            """, conn)

In [ ]:
#### los mejores calificados segun su fecha de visualizacion ###
pd.read_sql("""select fecha, title,
            avg(iif(rating = 0.0, Null, rating)) as avg_cal,
            count(iif(rating = 0.0, Null, rating)) as watch_num
            from ratingtitle
            group by fecha
            having watch_num >=15 and watch_num <=100
            order by avg_cal desc
            limit 20
            """, conn)